In [2]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import glob
import difflib
import warnings
warnings.filterwarnings("ignore")

In [79]:
def query_Azure(sql):
    
    server = '' 
    database = 'ELASTIC_MASTER' 
    username = '' 
    password = '' 
    AZURE = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
    conn = pyodbc.connect(AZURE)
    df = pd.read_sql(sql, conn)

    return df

sql = f"""
--sql
SELECT
 MBR.branch_name
,MBP.address
,MBP.city
,MBP.state1
,MBP.zipcode
,MBR.region_reporting
FROM [BI_MASTER].[MASTER_BRANCH] MBR
LEFT JOIN [BI_MASTER].[Master_Business_Partners_Distinct_DC] MBP
ON MBR.sap_vcode = MBP.sap_vcode
WHERE MBR.active = 'Y'
ORDER BY region_reporting
;
"""
store_db = query_Azure(sql)

In [93]:
store_db

,branch_name,address,city,state1,zipcode,region_reporting
0,Union,29-02 UNION ST,FLUSHING,NY,11354,R1
1,Northern 156,156-40 NORTHERN BLVD,FLUSHING,NY,11354,R1
2,Great Neck,495 GREAT NECK RD,GREAT NECK,NY,11021,R1
3,Williston Park,400 HILLSIDE AVE,WILLISTON PARK,NY,11596,R1
4,Hartsdale,371 N CENTRAL AVE,HARTSDALE,NY,10530,R1
...,...,...,...,...,...,...
75,Irvine3,3931 IRVINE BLVD,IRVINE,CA,92602,R8
76,Brookline,1026-1028 BEACON ST,BROOKLINE,MA,02446,R9
77,Quincy,101 FALLS BLVD,QUINCY,MA,02169,R9
78,Cambridge,581 MASS AVE,CAMBRIDGE,MA,02139,R9


In [326]:
# define time_periods to compare
def time_periods(n, m):
    # first day of current month
    first_day_of_month = datetime.now().replace(day=1)

    # most recent month, n months before, m months before
    current = (first_day_of_month + relativedelta(months=-1))
    tp_1 = (first_day_of_month + relativedelta(months=-n-1))
    tp_2 = (first_day_of_month + relativedelta(months=-m-1))

    tps = []
    for tp in [current, tp_1, tp_2]:
        tps.append(tp.strftime("%b"+" " +"%d") + " - "+ (tp + relativedelta(day=31)).strftime("%b"+" " +"%d"+", "+"%Y"))

    return tps


In [198]:
# load all raw data into one dictionary
def create_df(root, folder_names):
    raw_dict = {}
    for i in range(len(folder_names)):
        for file_path in glob.glob(root+"\\"+folder_names[i]+"\*.csv"):
            df = pd.read_csv(file_path)
            placer_id = df['entity_id'][0]

            if i == 0:
                raw_dict[placer_id] = [df, None, None]
            else:
                if placer_id in raw_dict.keys():
                    raw_dict[placer_id][i] = df
                    
    return raw_dict
        

In [321]:
# clean each dataframe in dictionary and return new dictionary with cleaned dataframes
def clean_dict(raw_dict):
    # load raw data dictionary
    temp_dict = raw_dict
    # create new dictionary to return
    cleaned_dict = {}
    for entity_id in temp_dict.keys():
        df_lst = temp_dict[entity_id]
        
        # combine three dataframes into one 
        current = df_lst[0] # current dataframe
        store_name = current['entity_name'][0] + '- '+ current['entity_address'][0]
        # filter dataframe
        current = current[(current['favorite_entity_type']=='venue')&(current['favorite_category']=='groceries')]
        # empty dataframe if last month or last year data is not available
        if df_lst[1] is None:
            last_month = pd.DataFrame({'favorite_entity_id':[],'#_of_visitors':[],'%_of_visitors':[]})
        else:
            last_month = df_lst[1]
            last_month = last_month[(last_month['favorite_entity_type']=='venue')&(last_month['favorite_category']=='groceries')][['favorite_entity_id','#_of_visitors', '%_of_visitors']]
        if df_lst[2] is None:
            last_year = pd.DataFrame({'favorite_entity_id':[],'#_of_visitors':[],'%_of_visitors':[]})
        else:
            last_year = df_lst[2]
            last_year = last_year[(last_year['favorite_entity_type']=='venue')&(last_year['favorite_category']=='groceries')][['favorite_entity_id','#_of_visitors', '%_of_visitors']]
        # left join
        new_df = pd.merge(current,last_month,how='left',on='favorite_entity_id',suffixes=('','_last_month'))
        new_df = pd.merge(new_df,last_year,how='left',on='favorite_entity_id',suffixes=('','_last_year'))
        # fill na with 0 when last year dat or last month data not available
        new_df['%_of_visitors_last_month'].fillna(0,inplace=True)
        new_df['%_of_visitors_last_year'].fillna(0,inplace=True)
        # caculate variance
        new_df['from_last_month'] = new_df['%_of_visitors']-new_df['%_of_visitors_last_month']
        new_df['from_last_year'] = new_df['%_of_visitors']-new_df['%_of_visitors_last_year']
        # create rank col
        new_df['rank'] = new_df['%_of_visitors'].rank(method='min',ascending=False).astype('int8')
        # drop irrelevant columns
        new_df = new_df[['entity_id','entity_name', 'entity_address','entity_lat', 'entity_lng', 'start_time','end_time',
                        'rank','favorite_entity_id','favorite_entity_name', 'address', 'city', 'lat', 'lng','distance_miles', 
                        '#_of_visitors', '%_of_visitors',
                        '#_of_visitors_last_month', '%_of_visitors_last_month','from_last_month',
                        '#_of_visitors_last_year','%_of_visitors_last_year','from_last_year']]
        
        # add new_df to dictionary
        cleaned_dict[store_name] = new_df

    return cleaned_dict


In [357]:

def visualize(cleaned_dict):
    token = 'pk.eyJ1IjoianVuZ3Nvb2xlZTEyMyIsImEiOiJjbG5pemliaWcwdDc4MmxtanlsazFjNnphIn0.DuoiXA_AYMRMk1ZQjzS6ZA'

            # loop all stores in dictionary
    for store in cleaned_dict.keys():
        try:
            # make subplots
            fig = make_subplots(
                rows=2,
                cols=2,
                specs = [[{"type": "mapbox","rowspan":2},{"type":"table"}],
                        [None, None]],
                subplot_titles=("","Top 10 Competitors List"),
                vertical_spacing=0.02,
                horizontal_spacing=0.04
                )

            # 1. map visualization
            df = cleaned_dict[store]
            # add h mart marker
            fig.add_trace(go.Scattermapbox(
                    lat=[df['entity_lat'][0]],
                    lon=[df['entity_lng'][0]],
                    mode='markers',
                    marker = go.scattermapbox.Marker(
                            size=20,color='green',
                            opacity=0.7),
                    text=",".join(store.split(",")[:-1]),
                    hoverinfo='text',
                    showlegend=False
                ),row=1,col=1)

            # add 10 competitors markers
            dfx = cleaned_dict[store][:10]
            color_scale = [(0, 'orange'), (1,'red')]
            fig.add_trace(go.Scattermapbox(
                    lat=dfx['lat'],
                    lon=dfx['lng'],
                    mode='markers+text',
                    marker=go.scattermapbox.Marker(
                        size=[x*500 for x in list(dfx["%_of_visitors"])],
                        color=[x*100 for x in list(dfx["%_of_visitors"])],
                        colorscale=color_scale,
                        colorbar=dict(title='Shared Customer (%)', x=-0.04),
                        opacity=0.7
                    ),
                    text=list(df['rank']),
                    name='',
                    hovertemplate= "Rank: " + dfx['rank'].astype(str) + "<br>" 
                        + "Name: " + dfx['favorite_entity_name'] + "<br>"
                        + "Shared Customer: " + (round(dfx['%_of_visitors']*100,2)).astype(str) +" %",
                    showlegend=False
                ),row=1,col=1)

            # 2. table visualization
            # dfx = dfx[['rank','favorite_entity_name','distance_miles','%_of_visitors','from_last_month','from_last_year']].round(2)
            fig.add_trace(go.Table( columnwidth=[10,60,10,10,10,10],
                                    header=dict(values=['Rank','Business Name','Distance', 'Shared Customer','From Last Month','From Last Year'],
                                                font_size=12),
                                    cells=dict(values= [[str(x) for x in list(dfx["rank"])],
                                                        [dfx["favorite_entity_name"][i] + " - " + dfx['address'][i] + ", " +dfx['city'][i] for i in range(len(dfx["favorite_entity_name"]))],
                                                        [str(round(x,2)) + "mi" for x in list(dfx["distance_miles"])],
                                                        [str(round(x*100,2)) + "%" for x in list(dfx['%_of_visitors'])],
                                                        ["(" + str(round(x*-100,2)) + "%)" if x <= 0 else str(round(x,2)) + "%" for x in list(dfx["from_last_month"])],
                                                        ["(" + str(round(x*-100,2)) + "%)" if x <= 0 else str(round(x,2)) + "%" for x in list(dfx["from_last_year"])]],
                                                font_color=['darkslategray',
                                                            ["green" if "H Mart" in x else "darkslategray" for x in list(dfx["favorite_entity_name"])],
                                                            'darkslategray','darkslategray', "darkslategray", "darkslategray"
                                                            ],
                                                height=30)
                ),row=1,col=2)

            fig.update_annotations(font_size=18,yshift=20)
            fig.update_layout(
                    mapbox=dict(accesstoken=token,zoom=11,style='light',center={'lat':df['entity_lat'][0],'lon':df['entity_lng'][0],}),
                    margin={"r":20,"t":100,"l":100,"b":10},
                    title = f'{store}',
                    title_font_size=24,
                    width=1600,height=1000
                    )
            fig.write_html(f"G:\Shared drives\Grand BK - Corporate Strategy\jungsoo\Placer_AI\Reports\Competitor_Visualizations\{store}.html") 
            print(f'Completed: {store}')
        except:
            print(f'Failed: {store}')

In [352]:
root = "G:\Shared drives\Grand BK - Corporate Strategy\jungsoo\Placer_AI\Datasets"
tps = time_periods(1,12)
folder_names = [f"Property_Favorite Places_Chains (Full) {tp}" for tp in tps]

raw_dict = create_df(root, folder_names)
cleaned_dict = clean_dict(raw_dict)
visualize(cleaned_dict)

Failed: H Mart- 3825 Alton Pkwy, Irvine, CA, United States
Failed: H Mart- 2600 Alton Pkwy, Irvine, CA, United States
